<a href="https://colab.research.google.com/github/TheBlackRus/liveproject_HumanPoseEstimation/blob/master/liveproject_humanpose_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the COCO dataset.

In [0]:
# This code downloads the coco dataset from Amazon S3 in parallel.
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import multiprocessing
import subprocess
#files = [ 'train2017.zip']#,'val2017.zip', 'annotations_trainval2017.zip']
files = [ 'val2017.zip', 'annotations_trainval2017.zip']

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
def download_and_unzip_from_s3(file_name, bucket_name='fast-ai-coco'):
    print("Downloading", file_name)
    s3.download_file(bucket_name, file_name, file_name)
    print("Finished downloading", file_name, ". Starting to unzip.")
    subprocess.run(["unzip", file_name])
    print("Finished unzipping", file_name)

# Download in parallel
num_cpus = multiprocessing.cpu_count()
with multiprocessing.Pool(num_cpus) as p:
    p.map(download_and_unzip_from_s3, files)

print("Done transferring all datasets")

Finished downloading annotations_trainval2017.zip . Starting to unzip.
Finished downloading val2017.zip . Starting to unzip.
Finished unzipping annotations_trainval2017.zip
Finished unzipping val2017.zip
Done transferring all datasets


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Generate the input images. 

In [0]:
# read the meta data of the validation set
import json
file_name = "annotations//person_keypoints_val2017.json"
with open(file_name, 'r') as json_raw:
    meta = json.load(json_raw)
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import math
from scipy.io import loadmat
from matplotlib.pyplot import imshow

In [0]:
import json
from torch.utils.data import Dataset
import numpy as np
%matplotlib inline
#from matplotlib.pyplot import imshow
from matplotlib import image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from scipy.ndimage import gaussian_filter

class HumanPoseDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, meta_path,train= False, transform=None):
        
        self.meta = self.load_meta(meta_path)
        self.train = train
        
        self.img_annotations = self.filter_annotations(self.meta["annotations"])
        #self.img_info = meta["images"]
        #self.img_catogory = meta["categories"][0]["keypoints"]
        self.mean = [0.485, 0.456, 0.406]
        self.std = [0.229, 0.224, 0.225]

    def load_meta(self,path):
      
      with open(path, 'r') as json_raw:
          meta = json.load(json_raw)
      return meta

    def filter_annotations(self,meta):
      min_width = 48 #192 # 12
      min_height = 64 # 256 # 16
      print("LEN before filter",len(meta))
      annot = filter(lambda x: x["iscrowd"] == 0 and 
                    x['bbox'][2]>= min_width and x['bbox'][3]>=min_height and
                    any(map(lambda y: y>0, x["keypoints"][2:-1:3])) # visible              
                    ,meta)
      annot = list(annot)
      """
      cleaned_annot = []
      for sample in meta:
        no_crowd = sample["iscrowd"] == 0
        start_x,start_y, w,h = sample['bbox']
        not_too_small = w>= min_width and h>=min_height
        inside_box = False
        #print(sample["keypoints"])
        keypoints = sample["keypoints"]
        for i in range(len(keypoints)//3):
          
          keypoint = (keypoints[i*3],keypoints[i*3+1])
          kx,ky = keypoint
          if not (start_x<kx and kx < (start_x+w) and start_y<ky and ky < (start_y+h)):
            inside_box = True
            break
        if all([no_crowd, not_too_small, inside_box]):
          cleaned_annot.append(sample)
        """
      print("AFTER", len(annot))
      return annot #cleaned_annot

    def __len__(self):
        #print(len(self.img_annotations), "anot")
        return len(self.img_annotations)

    def get_image_name(self,annot):
      
      img_file = str(annot["image_id"])
      img_file = img_file.zfill(12) +".jpg" #img_info["file_name"]
      if self.train:
        return "train2017//"+img_file
      else:   
        return "val2017//"+img_file

    def resize_image(self,img, target_width=192, target_height=256):
      img_resized = cv2.resize(img,(target_width,target_height))   
      return img_resized

    def crop_image(self,img, upper_left_corner, size):
      #print(upper_left_corner,size, img.shape)
      start_x, start_y = upper_left_corner
      w,h = size
      #print("CROP",int(start_y),(int(start_y+h-1)),int(start_x),int(start_x+w-1), img.shape)
      img_cropped = img[int(start_y):(int(start_y+h)),int(start_x):int(start_x+w),:]
      #print("CROP", img.shape)
      return img_cropped

    def adjust_keypoint(self,keypoint, start, scale):
      keypoint_x, keypoint_y= keypoint
      start_x,start_y = start
      sx,sy = scale
      x = (keypoint_x-start_x)*sx
      y = (keypoint_y-start_y)*sy
      return x,y

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        annot = self.img_annotations[idx]
        keypoints = annot["keypoints"]

        img_name = self.get_image_name(annot)
        image = io.imread(img_name)
        #
        #print("IDX",idx)
        if len(image.shape) <3:
          # thats a freakin gray image
          image = np.dstack((image, image,image))
        #plt.imshow(image)
        target_width = 192
        target_height = 256

        #print(annot["bbox"],"box")
        start_x,start_y, w,h = annot["bbox"]
        #print("box",annot["bbox"])
        #print(image.shape,"ishape")
        img_crop = self.crop_image(image, (start_x,start_y),(w,h))
        #print("crop",img_crop.shape)
        #plt.imshow(img_crop)
        img_crop = self.resize_image(img_crop,target_width,target_height)
        
        img = img_crop / 255.0
        img = (img-self.mean)/self.std
        # imgshape (256, 192, 3)
        #print("img",img.shape)
        img = np.transpose(img,[2,0,1]) # channel first

        #img_batch = torch.Tensor([img, img])
        #print(img_batch.shape)
        #img_batch = img_batch.permute([0,3,2,1])

        sx = target_width/w 
        sy = target_height/h
        #print("sxsy",sx,sy)

        validity = annot["keypoints"][2::3]

        validity = np.array(validity)
        validity = validity > 0
        validity = validity.astype(np.int)
        #print(validity.shape)

        heatmaps = np.zeros((17,64,48))

        keypoints_adjusted = []
        for i in range(len(keypoints)//3):
          
          keypoint = (keypoints[i*3],keypoints[i*3+1])
          visible = keypoints[i*3+2]
          #print(keypoint)
          if visible:
            x,y = self.adjust_keypoint(keypoint,(start_x,start_y),(sx,sy))
            #print("XY", x,y,int(y//4),int(x//4),sx,sy )
            #not (start_x<kx and kx < (start_x+w) and start_y<ky and ky < (start_y+h))
            kx , ky = keypoint
            #if kx< start_x or ky < start_y or kx> (start_x+w) or ky > (start_y +h):
            if not (start_x<kx and kx < (start_x+w) and start_y<ky and ky < (start_y+h)):
              validity[i] = 0
              #print("FAIL", start_x, kx, start_x+w, start_y,y, start_y+h)
            else:
              
              #if (y//4 >= 64 or x//4 >=47):
              #  print("fail",idx,x,y)
              heatmaps[i,int(y//4),int(x//4)] = 1
              heatmaps[i,:,:] =  gaussian_filter(heatmaps[i,:,:], sigma=2)
              #print(heatmaps[i,:,:].min(),heatmaps[i,:,:].max())
              heatmaps[i,:,:] -= heatmaps[i,:,:].min() 
              heatmaps[i,:,:] /= heatmaps[i,:,:].max()
            #print("N",heatmaps[i,:,:].min(),heatmaps[i,:,:].max())
            #plt.matshow(heatmaps[i,:,:])
            #print(meta["categories"][0]["keypoints"][i])
          else:
            x,y = (-1,-1)
          keypoints_adjusted.append(x)
          keypoints_adjusted.append(y)
          
        """
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks])
        landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)
        """
        return torch.Tensor(img), torch.Tensor(heatmaps), torch.Tensor(validity)

In [0]:
data_val = HumanPoseDataset(meta_path="annotations//person_keypoints_val2017.json")
data_train = HumanPoseDataset(meta_path="annotations//person_keypoints_train2017.json", train=True)
#data_train = HumanPoseDataset(meta_path="annotations//person_keypoints_val2017.json", train=False)

LEN before filter 11004
AFTER 4795
LEN before filter 262465
AFTER 112208


In [0]:
for i in range(len(data_val)):
  print(i)
  _, labels, _ = data_val[i]
  
  # h in range(labels.shape[0]):
  #  plt.matshow(labels[h,:,:])

979 anot
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
2

In [0]:
data_val[1]

(tensor([[[ 2.1462,  2.1462,  2.1462,  ...,  1.9749,  1.9407,  1.9749],
          [ 2.1119,  2.1119,  2.1290,  ...,  1.9749,  1.9064,  1.9749],
          [ 2.1119,  2.1119,  2.1119,  ...,  2.0092,  1.9578,  2.0092],
          ...,
          [-1.0733, -0.8335, -0.8335,  ...,  2.1633,  2.1633,  2.1462],
          [-0.7137, -0.9192, -1.2959,  ...,  2.1462,  2.1119,  2.1633],
          [-1.6727, -1.4672, -1.3644,  ...,  2.0605,  2.0263,  2.1462]],
 
         [[ 2.2710,  2.2710,  2.2710,  ...,  1.9909,  1.9734,  1.9734],
          [ 2.2360,  2.2360,  2.2535,  ...,  1.9909,  1.9559,  1.9909],
          [ 2.2360,  2.2360,  2.2360,  ...,  2.0259,  1.9909,  2.0084],
          ...,
          [-1.2129, -0.9678, -0.8452,  ...,  2.2185,  2.2185,  2.2185],
          [-1.1429, -1.2129, -1.2654,  ...,  2.2535,  2.2360,  2.2360],
          [-1.8081, -1.6506, -1.3179,  ...,  2.1660,  2.1660,  2.2010]],
 
         [[ 2.3611,  2.3611,  2.3611,  ...,  2.0474,  2.0300,  2.0300],
          [ 2.3263,  2.3263,

In [0]:
meta.keys()
print(meta["categories"])

[{'supercategory': 'person', 'id': 1, 'name': 'person', 'keypoints': ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle'], 'skeleton': [[16, 14], [14, 12], [17, 15], [15, 13], [12, 13], [6, 12], [7, 13], [6, 7], [6, 8], [7, 9], [8, 10], [9, 11], [2, 3], [1, 2], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7]]}]


In [0]:
example = 0
img_info = meta["images"][example]
img_annotation = meta["annotations"][example]
img_catogory = meta["categories"][0]["keypoints"]

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #block
        self.conv_1 = nn.Conv2d(in_channels = 3, out_channels=64,kernel_size=7,stride=2,padding=3)
        #nn.init.normal_(self.conv_1.weight, std=0.001)
        #nn.init.constant_(self.conv_1.bias,0)
        self.batch_norm_1 = nn.BatchNorm2d(64) # same as out_channels before layer 
        self.relu_1 = nn.ReLU()
        self.pool_1 = nn.MaxPool2d(2, 2)


        self.conv_2 = nn.Conv2d(in_channels = 64, out_channels=128,kernel_size=5,stride=1,padding=2)
        #nn.init.normal_(self.conv_2.weight, std=0.001)
        #nn.init.constant_(self.conv_2.bias,0)
        self.batch_norm_2 = nn.BatchNorm2d(128) # same as out_channels before layer 
        self.relu_2 = nn.ReLU()
        self.pool_2 = nn.MaxPool2d(2, 2)

        self.conv_3 = nn.Conv2d(in_channels= 128, out_channels=256,kernel_size=5,stride=1,padding=2)
        #nn.init.normal_(self.conv_3.weight, std=0.001)
        #nn.init.constant_(self.conv_3.bias,0)
        self.batch_norm_3 = nn.BatchNorm2d(256) # same as out_channels before layer 
        self.relu_3 = nn.ReLU()
        self.pool_3 = nn.MaxPool2d(2, 2)

        # lazy explanition in the workflow...
        # right before
        self.up1 = nn.ConvTranspose2d(256, 256, 4, stride=2,padding=1)
        nn.init.normal_(self.up1.weight, std=0.001)
        self.batch_norm_4 = nn.BatchNorm2d(256) # same as out_channels before layer 
        self.relu_4 = nn.ReLU()

        self.up2 = nn.ConvTranspose2d(256, 256, 4, stride=2,padding=1)
        nn.init.normal_(self.up2.weight, std=0.001)
        self.batch_norm_5 = nn.BatchNorm2d(256) # same as out_channels before layer 
        self.relu_5 = nn.ReLU()

        self.fc1 = nn.Conv2d(in_channels= 256, out_channels=17,kernel_size=1,stride=1,padding=0) #nn.Linear(24 * 4 * 4, 10)# 24 chans x 32//(2*2*2)
        self.sig = nn.Sigmoid()
        
        #nn.init.normal_(self.fc1.weight, std=0.001)
        #nn.init.constant_(self.fc1.bias,0)

        

    def forward(self, x):
        x = self.conv_1(x)
        x = self.batch_norm_1(x)
        x = self.relu_1(x)
        x = self.pool_1(x)

        x = self.conv_2(x)
        x = self.batch_norm_2(x)
        x = self.relu_2(x)
        x = self.pool_2(x)

        x = self.conv_3(x)
        x = self.batch_norm_3(x)
        x = self.relu_3(x)
        x = self.pool_3(x)

        x = self.up1(x)
        x = self.batch_norm_4(x)
        x = self.relu_4(x)

        x = self.up2(x)
        x = self.batch_norm_5(x)
        x = self.relu_5(x)
        



        x = self.fc1(x)#x.view(-1, 24 * 4 * 4)
        x = self.sig(x)
        
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        #x = self.fc1(x)
        return x



net = Net()
print(net)

Net(
  (conv_1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_1): ReLU()
  (pool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_2): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch_norm_2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_2): ReLU()
  (pool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_3): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch_norm_3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_3): ReLU()
  (pool_3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (up1): ConvTranspose2d(256, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (batch_norm_4): BatchNorm2d(256, eps=1e-05, momentum=0.1,

In [0]:
#from torchsummary import summary
#summary(net, (3, 256, 192))

In [0]:
img, heatmaps, validity = data_val[0]

In [0]:
def my_loss(output, target,validity):
    #print("out",output.shape)
    #print("tar",target.shape)
    #print("val",validity.shape)
    """
    for b in range(output.shape[0]):
      for i in range(len(validity[b])):
        torch.
        output[b,i,:,:] = output[b,i,:,:] *validity[b,i]
        target[b,i,:,:] = target[b,i,:,:] *validity[b,i]

    loss = torch.mean((output - target)**2)
    """
    #print("-"*70)
    #print((output-target).shape)
    #print("x"*50)
    #print(validity.shape)
     
    loss = torch.mean(((output - target))**2,(2,3))
    #print("los",loss.shape)
    #print("val",(loss*validity).shape)
    loss = torch.mean( torch.mul(  loss,validity))
    
    
    return loss

In [0]:
def get_max_preds(batch_heatmaps):
    '''
    get predictions from score maps
    heatmaps: numpy.ndarray([batch_size, num_joints, height, width])
    '''
    #assert isinstance(batch_heatmaps, np.ndarray), \
    #    'batch_heatmaps should be numpy.ndarray'
    assert batch_heatmaps.ndim == 4, 'batch_images should be 4-ndim'

    batch_size = batch_heatmaps.shape[0]
    num_joints = batch_heatmaps.shape[1]
    width = batch_heatmaps.shape[3]
    heatmaps_reshaped = batch_heatmaps.reshape((batch_size, num_joints, -1))
    idx = np.argmax(heatmaps_reshaped, 2)
    maxvals = np.amax(heatmaps_reshaped, 2)

    maxvals = maxvals.reshape((batch_size, num_joints, 1))
    idx = idx.reshape((batch_size, num_joints, 1))

    preds = np.tile(idx, (1, 1, 2)).astype(np.float32)

    preds[:, :, 0] = (preds[:, :, 0]) % width
    preds[:, :, 1] = np.floor((preds[:, :, 1]) / width)

    pred_mask = np.tile(np.greater(maxvals, 0.0), (1, 1, 2))
    pred_mask = pred_mask.astype(np.float32)

    preds *= pred_mask
    return preds, maxvals


def calc_dists(preds, target, normalize):
    preds = preds.astype(np.float32)
    target = target.astype(np.float32)
    dists = np.zeros((preds.shape[1], preds.shape[0]))
    for n in range(preds.shape[0]):
        for c in range(preds.shape[1]):
            if target[n, c, 0] > 1 and target[n, c, 1] > 1:
                normed_preds = preds[n, c, :] / normalize[n]
                normed_targets = target[n, c, :] / normalize[n]
                dists[c, n] = np.linalg.norm(normed_preds - normed_targets)
            else:
                dists[c, n] = -1
    return dists


def dist_acc(dists, thr=0.5):
    ''' Return percentage below threshold while ignoring values with a -1 '''
    dist_cal = np.not_equal(dists, -1)
    num_dist_cal = dist_cal.sum()
    if num_dist_cal > 0:
        return np.less(dists[dist_cal], thr).sum() * 1.0 / num_dist_cal
    else:
        return -1

def calc_accuracy(output, target, hm_type='gaussian', thr=0.5):
    '''
    Calculate accuracy according to PCK,
    but uses ground truth heatmap rather than x,y locations
    First value to be returned is average accuracy across 'idxs',
    followed by individual accuracies
    '''
    idx = list(range(output.shape[1]))
    norm = 1.0
    if hm_type == 'gaussian':
        pred, _ = get_max_preds(output)
        target, _ = get_max_preds(target)
        h = output.shape[2]
        w = output.shape[3]
        norm = np.ones((pred.shape[0], 2)) * np.array([h, w]) / 10
    dists = calc_dists(pred, target, norm)

    acc = np.zeros((len(idx) + 1))
    avg_acc = 0
    cnt = 0

    for i in range(len(idx)):
        acc[i + 1] = dist_acc(dists[idx[i]])
        if acc[i + 1] >= 0:
            avg_acc = avg_acc + acc[i + 1]
            cnt += 1

    avg_acc = avg_acc / cnt if cnt != 0 else 0
    if cnt != 0:
        acc[0] = avg_acc
    return avg_acc
def get_accuracy(model, dataloader):

    model.eval()
    # TODO: calculate accuracy
    acc = 0
    count = 0
    for i, data in enumerate(dataloader, 0):
      inputs, labels,validity = data
      #labels = labels.squeeze()
      inputs = inputs.cuda()
     
      
      outputs = net(inputs)
      
      #print(predicted, labels)
      o = outputs.cpu().detach().numpy()[:,:,:,:]
      l = labels.detach().numpy()[:,:,:,:]
      #plt.matshow(o[0,0,:,:])
      #print(l.type)
      a = calc_accuracy(o,l)
      #print("a",a)
      acc +=a
      count = count + 1
    #print("AC",acc,count)
    #acc = acc.to(torch.float)
    
    if count >0:
      accuracy = acc/count
    else:
      accuracy = -1
    #outputs = net(inputs)
    model.train()
    
    return accuracy

In [0]:
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR

criterion = my_loss
optimizer = optim.Adam(net.parameters(),lr=0.0001) # ,lr=0.001 #optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

scheduler = MultiStepLR(optimizer, milestones=[10,100], gamma=0.1)

trainloader = torch.utils.data.DataLoader(data_train, batch_size=32,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(data_val, batch_size=32,
                                          shuffle=False, num_workers=2)

In [0]:
print("device","cuda" if torch.cuda.is_available() else "cpu")
net.cuda()
for epoch in range(100):  # loop over the dataset multiple times
    print("EPOCH",epoch)
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        #print("-"*60)
        #print("datalen",len(data))
        inputs, labels, validity = data
        #print(labels.shape)
        #for b in range(labels.shape[0]):
        #  for h in range(labels.shape[1]):
        #    plt.matshow(labels[b,h,:,:])
        #print("L",labels.shape)
        #labels = labels.squeeze()
        #print("l",labels.shape)
        inputs = inputs.cuda()
        labels = labels.cuda()
        validity = validity.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        outputs.cuda()
        #print(labels,labels.shape)
        loss = criterion(outputs, labels,validity)
        loss.backward()
        optimizer.step()
        

        # print statistics
        running_loss += loss.item()
        #print("#"*60)
        #if i % 100 == 10:    # print every 100 mini-batches
    scheduler.step()
    print('[%d, %5d] loss: %.7f' %(epoch + 1, i + 1, running_loss ))
    acc = get_accuracy(net,testloader)
    print("acc %.14f" % acc)
    running_loss = 0.0

print('Finished Training')

device cuda
EPOCH 0
[1,  3507] loss: 20.2735727
acc 0.32071281320975
EPOCH 1
[2,  3507] loss: 7.0666584
acc 0.38308203489505
EPOCH 2
[3,  3507] loss: 6.6478848
acc 0.41898041011279
EPOCH 3
[4,  3507] loss: 6.4030771
acc 0.44165528246110
EPOCH 4
[5,  3507] loss: 6.2310237
acc 0.45107220395456
EPOCH 5
[6,  3507] loss: 6.0975563
acc 0.46454900812868
EPOCH 6
[7,  3507] loss: 5.9905730
acc 0.47069109636303
EPOCH 7
[8,  3507] loss: 5.8975002
acc 0.47390906441757
EPOCH 8
[9,  3507] loss: 5.8195777
acc 0.48016981872986
EPOCH 9
[10,  3507] loss: 5.7479080
acc 0.48335761313486
EPOCH 10
[11,  3507] loss: 5.6838441
acc 0.48287028994149
EPOCH 11
[12,  3507] loss: 5.6268232
acc 0.48592820942091
EPOCH 12
[13,  3507] loss: 5.5722530
acc 0.48655302126171
EPOCH 13
[14,  3507] loss: 5.5237039
acc 0.48837847373525
EPOCH 14
[15,  3507] loss: 5.4781577
acc 0.48782826775548
EPOCH 15
[16,  3507] loss: 5.2411255
acc 0.49711092004856
EPOCH 16
[17,  3507] loss: 5.1980484
acc 0.49547665006097
EPOCH 17
[18,  3507]